In [1]:
import json
import numpy as np
import pandas as pd

In [2]:
with open('analysis_dataset.json', 'r') as js:
     games = json.load(js)

In [3]:
gameId = '4733146699'
games[gameId]

{'win': {'100': 'Win', '200': 'Fail'},
 'ban': {'100': [89, 104, 126, 91, 360], '200': [142, 25, 117, 80, 91]},
 'pick': {'100': [76, 497, 21, 54, 777], '200': [60, 236, 29, 40, 114]},
 'pick_role': {'100': {'JUNGLE_NONE': 76,
   'BOTTOM_DUO_SUPPORT': 497,
   'BOTTOM_DUO_CARRY': 21,
   'TOP_SOLO': 54,
   'MIDDLE_SOLO': 777},
  '200': {'JUNGLE_NONE': 60,
   'MIDDLE_SOLO': 236,
   'BOTTOM_DUO_CARRY': 29,
   'BOTTOM_DUO_SUPPORT': 40,
   'TOP_SOLO': 114}}}

In [4]:
df_champion = pd.read_csv('../../data/csv/champion.csv')
df_champion = df_champion.drop(['Unnamed: 0'], axis=1)
column = ['champion','championId']
df_champion.tail()

,id,key,line
147,Zed,238,2
148,Ziggs,115,2
149,Zilean,26,4
150,Zoe,142,2
151,Zyra,143,4


In [5]:
champion_id_list = list(df_champion.key)
champion_id_list.sort()

# Prior probability( = P(H))

In [6]:
def safe_divide(x, y):
    """
    prevent from dividing by 0
    :param x: numerator
    :param y: denominator
    :return: float
    """
    try:
        return round(x / y, 4)
    except ZeroDivisionError:
        return 0

# Champion win rate( = P(H))

In [7]:
def champion_win_rate(champion_id):
    """
    챔피언 하나의 승률.
    :param champion_id: championId
    :return: [승률, 챔피언이 플레이한 게임수, 챔피언이 플레이해서 이긴 게임수]
    """
    game_count = 0
    win_count = 0
    teamIds = ['100', '200']
    for gameId in games.keys():
        for teamId in teamIds:
            if champion_id in games[gameId]['pick'][teamId]:
                game_count += 1
                if games[gameId]['win'][teamId] == 'Win':
                    win_count += 1
    return [safe_divide(win_count, game_count), game_count, win_count]

In [8]:
champion_win_rate(498)

[0.5397, 63, 34]

In [9]:
champion_win_rates = pd.DataFrame()
for champion_id in champion_id_list:
    champion_win_rates = champion_win_rates.append([[champion_id, 
                                                     champion_win_rate(champion_id)[0],
                                                     champion_win_rate(champion_id)[1],
                                                     champion_win_rate(champion_id)[2]]])
champion_win_rates.columns = ['champion_id', 'champion_win_rate', 'game_count', 'champion_win_count']
champion_win_rates.reset_index(inplace=True)
del champion_win_rates['index']

In [10]:
champion_win_rates.sort_values('champion_win_rate', ascending=False)

,champion_id,champion_win_rate,game_count,champion_win_count
137,420,1.0000,3,3
44,45,0.7500,4,3
33,34,0.7143,7,5
84,101,0.7000,10,7
51,56,0.6290,186,117
...,...,...,...,...
32,33,0.2500,12,3
35,36,0.2500,8,2
36,37,0.1429,7,1
16,17,0.0000,2,0


# Evidence( = P(E))
## P(E) = P(E|H) P(H) + P(E|~ H) P(~ H)



In [18]:
def without_pick_champion_win_rate(my_champion ,my_team_champions, opponent_team_champions):
    """
    내가 고른 챔피언이 없는 게임들 중에서 우리 팀과 적 팀이 고른 챔피언이 속한 게임의 승률.
    항상 우리 팀이 고른 챔피언 숫자가 적을 경우 픽하는 상황으로 한정
    실 게임에서는 동시에 2명이 픽 하는 경우, 다른 summoner가 먼저 픽 할 경우 고른 챔피언 숫자가 우리팀과 적팀이 같으나 여기선 무시
    :param my_champion: 내가 고른 championId int
    :param my_team_champions: 우리 팀의 championId list
    :param opponent_team_champions: 적 팀의 championId list
    :return: [승률, 플레이한 게임수, 플레이해서 이긴 게임수]
    """
    # 같은팀 두명이 동시에 고르는 경우 내가 픽 했더라도 팀원은 아직 고르지 않았다는 가정
    if my_champion in my_team_champions:
        print('다른 챔피언을 선택하세요. 이미 픽된 챔피언입니다.')
        return [0, 0, 0]
    else:
        if my_champion in opponent_team_champions:
            print('다른 챔피언을 선택하세요. 이미 픽된 챔피언입니다.')
            return [0, 0, 0]
        else:
            teamIds = ['100', '200']
            for gameId in games.keys():
                # 내가 고른 챔피언이 없는 경우만 뽑음
                if champion_id not in games[gameId]['pick'][teamIds[0]] + games[gameId]['pick'][teamIds[1]]:
                    if (len(my_team_champions), len(opponent_team_champions)) in [(0,1),(1,2),(2,3),(3,4),(4,5)]:
                        game_count = 0
                        win_count = 0
                        for gameId in games.keys():
                            for teamId in teamIds:
                                # 우리 팀이 뽑은 챔피언이 games안에 모두 있는지 확인
                                if len(set(games[gameId]['pick'][teamId]) - set(my_team_champions)) == (
                                    5 - len(my_team_champions)):
                                    # 적 팀이 뽑은 챔피언이 반대 팀에 있는지 확인
                                    if len(set(games[gameId]['pick'][list(set(teamIds) - set([teamId]))[0]]) - 
                                           set(opponent_team_champions)) == (5 - len(opponent_team_champions)):
                                        # 우리 팀과 적 팀이 뽑은 챔피언이 모두 있는 경우는 경기수 증가
                                        game_count += 1
                                        if games[gameId]['win'][teamId] == 'Win':
                                            # 우리 팀이 이긴 경우 승수 추가
                                            win_count += 1
                        return [safe_divide(win_count, game_count), game_count, win_count]
                    else:
                        return print('''올바른 Input 값을 넣으세요. 
Only (len(my_team_champions), len(opponent_team_champions)) in [(0,1), (1,2), (2,3), (3,4), (4,5)] is allowed.''')

In [13]:
without_pick_champion_win_rate('498', [], [59])

[0.5366, 82, 44]

In [15]:
without_pick_champion_win_rate('498', [24, 56], [59, 110])

Input correct value. 
Only (len(my_team_champions), len(opponent_team_champions)) in [(0,1), (1,2), (2,3), (3,4), (4,5)] is allowed.


In [21]:
without_pick_champion_win_rate('498', [24], [59, 110])
# 한계..

[0, 0, 0]

# likelihood( = P(E|H))

In [20]:
def pick_champion_win_rate(my_champion ,my_team_champions, opponent_team_champions):
    """
    내가 고른 챔피언이 있는 게임들 중에서 우리 팀과 적 팀이 고른 챔피언이 속한 게임의 승률
    항상 우리 팀이 고른 챔피언 숫자가 적을 경우 픽하는 상황으로 한정
    실 게임에서는 동시에 2명이 픽 하는 경우, 다른 summoner가 먼저 픽 할 경우 고른 챔피언 숫자가 우리팀과 적팀이 같으나 여기선 무시
    :param my_champion: 내가 고른 championId int
    :param my_team_champions: 우리 팀의 championId list
    :param opponent_team_champions: 적 팀의 championId list
    :return: [승률, 플레이한 게임수, 플레이해서 이긴 게임수]
    """
    # 같은팀 두명이 동시에 고르는 경우 내가 픽 했더라도 팀원은 아직 고르지 않았다는 가정
    if my_champion in my_team_champions:
        print('다른 챔피언을 선택하세요. 이미 픽된 챔피언입니다.')
        return [0, 0, 0]
    else:
        if my_champion in opponent_team_champions:
            print('다른 챔피언을 선택하세요. 이미 픽된 챔피언입니다.')
            return [0, 0, 0]
        else:
            if (len(my_team_champions) + 1, len(opponent_team_champions)) in [(1,1),(2,2),(3,3),(4,4),(5,5)]:
                new_my_team_champions = my_team_champions + [my_champion]
                game_count = 0
                win_count = 0
                teamIds = ['100', '200']
                for gameId in games.keys():
                    for teamId in teamIds:
                        # 내가 새로 뽑을 챔피언과 우리 팀이 뽑은 챔피언이 games안에 모두 있는지 확인
                        if len(set(games[gameId]['pick'][teamId]) - set(new_my_team_champions)) == \
                            (5 - len(new_my_team_champions)):
                            # 적 팀이 뽑은 챔피언이 반대 팀에 있는지 확인
                            if len(set(games[gameId]['pick'][list(set(teamIds) - set([teamId]))[0]]) - 
                                   set(opponent_team_champions)) == (5 - len(opponent_team_champions)):
                                # 우리 팀과 적 팀이 뽑은 챔피언이 모두 있는 경우는 경기수 증가
                                game_count += 1
                                if games[gameId]['win'][teamId] == 'Win':
                                    # 우리 팀이 이긴 경우 승수 추가
                                    win_count += 1
                return [safe_divide(win_count, game_count), game_count, win_count]
            else:
                return print('''올바른 Input 값을 넣으세요.
Only (len(my_team_champions), len(opponent_team_champions)) in [(0,1), (1,2), (2,3), (3,4), (4,5)] is allowed.''')

In [32]:
pick_champion_win_rate(498, [], [240])

[0, 0, 0]

In [27]:
pick_champion_win_rate(90, [113, 90, 63], [59, 110, 80, 134])

다른 챔피언을 선택하세요. 이미 픽된 챔피언입니다.


[0, 0, 0]

In [28]:
pick_champion_win_rate(59, [113, 90, 63], [59, 110, 80, 134])

다른 챔피언을 선택하세요. 이미 픽된 챔피언입니다.


[0, 0, 0]

In [29]:
pick_champion_win_rate(59, [113, 90, 63, 100], [110, 80, 134])

올바른 Input 값을 넣으세요.
Only (len(my_team_champions), len(opponent_team_champions)) in [(0,1), (1,2), (2,3), (3,4), (4,5)] is allowed.


# Posterior probability ( = P(H|E))

## Expect win rate

In [33]:
def expect_win_rate(my_champion, my_team_champions, opponent_team_champions):
    """
    내가 특정 챔피언을 고를 경우 우리 팀과 적 팀이 선택한 챔피언들을 바탕으로 예상되는 승률
    :param my_champion: 내가 고른 championId int
    :param my_team_champions: 우리 팀의 championId list
    :param opponent_team_champions: 적 팀의 championId list
    :return: 예상 승률
    """
    # P(E|H) * P(H)
    upper = (pick_champion_win_rate(my_champion, my_team_champions, opponent_team_champions)[0] * 
             champion_win_rate(my_champion)[0])
    # P(E) = P(E|H) * P(H) + P(E|~H) * P(~H)
    lower = (pick_champion_win_rate(my_champion, my_team_champions, opponent_team_champions)[0] * 
             champion_win_rate(my_champion)[0]) + \
            (without_pick_champion_win_rate(my_champion, my_team_champions, opponent_team_champions)[0] *      
            (1 - champion_win_rate(my_champion)[0]))
    if np.logical_or(lower==0, upper==0):
        return print('계산불가. 샘플수 부족...')
    else:
        # P(H|E)= P(E|H) * P(H) / P(E)
        return safe_divide(upper, lower)

In [34]:
my_champion = 59
my_team_champions = [18]
opponent_team_champions = [498, 81]
print('prior probability:', champion_win_rate(my_champion)[0])
print('likelihood:', pick_champion_win_rate(my_champion, my_team_champions, opponent_team_champions)[0])
print('evidence:', round((pick_champion_win_rate(my_champion, my_team_champions, opponent_team_champions)[0] * 
                    champion_win_rate(my_champion)[0]) + \
                   (without_pick_champion_win_rate(my_champion, my_team_champions, opponent_team_champions)[0] *      
                    (1 - champion_win_rate(my_champion)[0])), 4))
print('posterior probability:', expect_win_rate(my_champion, my_team_champions, opponent_team_champions))

prior probability: 0.4634
likelihood: 0
evidence: 0.0
계산불가. 샘플수 부족...
posterior probability: None


# Recommend champion